In [0]:
from utils.logs import print_args
from utils.spark_delta import merge, table_exists

In [0]:
class UsersBronzeETL:
    def __init__(self, spark, dt_start, dt_end, pk=None):
        self.spark = spark
        self.dt_start = dt_start
        self.dt_end = dt_end
        self.pk = self._get_default_pk(pk)
        self.default_date = '1901-01-01'
        
    @staticmethod
    def _get_default_pk(pk):
        if pk is None:
            return ['id_oid']
        return pk
        
    @print_args(print_kwargs=['source_tb'])
    def extract(self, source_tb):
        latest_update = f"COALESCE(updated_at_date, created_at_date, TIMESTAMP '{self.default_date}')"
        df = spark.sql(f"""
        WITH latest AS (
            SELECT 
                {','.join(self.pk)}
                , MAX({latest_update}) AS latest_update
            FROM {source_tb}
            WHERE {latest_update} BETWEEN '{self.dt_start}' AND '{self.dt_end}'
            GROUP BY {','.join(self.pk)}
        )
        SELECT s.* FROM {source_tb} s
        JOIN latest l
            ON {" AND ".join([f"s.{c}=l.{c}" for c in self.pk])}
            AND {latest_update}=l.latest_update
        WHERE {latest_update} BETWEEN '{self.dt_start}' AND '{self.dt_end}'
        """)
        return df
    
    def transform(self):
        pass
    
    @print_args(print_kwargs=['target_tb', 'drop'])
    def load(self, df, target_tb, drop=False):
        if drop and table_exists(target_tb, self.spark):
            print(f"Dropping table {target_tb}")
            self.spark.sql(f"DROP TABLE {target_tb}").show()

        merge(df, target_tb, self.pk, spark_session=self.spark)

In [0]:
%run ./etl_constants

In [0]:
etl = UsersBronzeETL(spark, DT_START, DT_END)
df = etl.extract(source_tb=TARGET_USERS_RAW_TB)
etl.load(df, target_tb=TARGET_USERS_BRONZE_TB, drop=DROP_BRONZE)

In [0]:
df.count()

In [0]:
spark.sql(f"SELECT * FROM {TARGET_USERS_BRONZE_TB}").count()